In [2]:
import pandas as pd
import s3fs
import numpy as np
import swifter

In [ ]:
def f(lang, embeddings, index):
    if index % 1000000  == 0:
        print(index)
    return [f'{lookup[lang]+i}:{e}' for i, e in enumerate(embeddings)]

In [ ]:
lookup = {'ko': 40578728,
 'en': 40578748,
 'id': 40578768,
 'pt': 40578788,
 'de': 40578808,
 'ja': 40578828,
 'fr': 40578848,
 'ru': 40578868,
 'fa': 40578888,
 'ar': 40578908,
 'tr': 40578928,
 'te': 40578948,
 'vi': 40578968,
 'tl': 40578988,
 'el': 40579008,
 'ur': 40579028,
 'it': 40579048,
 'et': 40579068,
 'ta': 40579088,
 'mr': 40579108,
 'ca': 40579128,
 'af': 40579148,
 'fi': 40579168,
 'gu': 40579188,
 'he': 40579208,
 'hr': 40579228,
 'mk': 40579248,
 'hu': 40579268,
 'ml': 40579288,
 'kn': 40579308,
 'bn': 40579328,
 'lv': 40579348,
 'pa': 40579368,
 'sw': 40579388,
 'bg': 40579408}

In [ ]:
new_names = list(range(1,21))
new_names = list(map(str, new_names))

In [ ]:
all_features = ["language"]
train = pd.read_csv("s3://recsys-challenge-2020/training.tsv", encoding="utf-8",
                    names = all_features, usecols= [7], sep="\x01"
                   )
print('train language loaded')

In [ ]:
all_lang_output_train = pd.read_csv('s3://recsys-challenge-2020/all_lang_output_train.csv')
lang_embedding_train = pd.concat([all_lang_output_train[['sentence_embedding']], train[['language']]], axis = 1)
lang_embedding_train.sentence_embedding = lang_embedding_train.sentence_embedding.apply(lambda x: np.fromstring(x[1:-1], sep=' '))

In [ ]:
for chunk in np.array_split(lang_embedding_train,20):
    chunk_embeddings_list = pd.DataFrame()
    chunk_embeddings_list = chunk.swifter.apply(lambda x: f(x.language, x.sentence_embedding, x.name), axis=1, result_type='expand')
    chunk_embeddings_list.to_csv('lang_embedd_train.csv', header = False, index = False, mode = 'a', sep = ' ')